<a href="https://colab.research.google.com/github/jihyeon602/DeepLearning_practice/blob/main/%EB%B0%98%EB%A0%A4%EB%8F%99%EB%AC%BC_%EC%A7%88%EB%B3%91_%EC%A7%84%EB%8B%A8%EC%9D%84_%EC%9C%84%ED%95%9C_%EC%98%81%EC%83%81_%EB%8D%B0%EC%9D%B4%ED%84%B0_(%EA%B7%BC%EA%B3%A8%EA%B2%A9%EA%B3%84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 파일 로드

In [1]:
!pip install -U gdown

!gdown 11SIkR8k0sUVKF9t73HcVo5Ib_rNjLLWD --output VS1_CAT.zip

Downloading...
From (original): https://drive.google.com/uc?id=11SIkR8k0sUVKF9t73HcVo5Ib_rNjLLWD
From (redirected): https://drive.google.com/uc?id=11SIkR8k0sUVKF9t73HcVo5Ib_rNjLLWD&confirm=t&uuid=e6ebd024-2246-4cdd-9e96-a0396d07f130
To: /content/VS1_CAT.zip
100% 275M/275M [00:03<00:00, 72.1MB/s]


In [2]:
!gdown 1qZz9TX7PmIyuXCxv-4yLnXtL52zrGpCY --output TS1_CAT.zip

Downloading...
From (original): https://drive.google.com/uc?id=1qZz9TX7PmIyuXCxv-4yLnXtL52zrGpCY
From (redirected): https://drive.google.com/uc?id=1qZz9TX7PmIyuXCxv-4yLnXtL52zrGpCY&confirm=t&uuid=006c0437-7ba6-4e1b-9a38-837f586bcb71
To: /content/TS1_CAT.zip
100% 2.37G/2.37G [00:24<00:00, 95.0MB/s]


In [3]:
!gdown 1kbXp-wI1dlX_9TgPxG9juScrOoBUf5l6 --output TL1_CAT.zip
!gdown 1tawhyY-iN3rzw99NZ9678gG-dioYKtmH --output VL1_CAT.zip

Downloading...
From (original): https://drive.google.com/uc?id=1kbXp-wI1dlX_9TgPxG9juScrOoBUf5l6
From (redirected): https://drive.google.com/uc?id=1kbXp-wI1dlX_9TgPxG9juScrOoBUf5l6&confirm=t&uuid=6b1cd509-7fe3-45a6-9ed4-3c69def18034
To: /content/TL1_CAT.zip
100% 6.23M/6.23M [00:00<00:00, 118MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tawhyY-iN3rzw99NZ9678gG-dioYKtmH
To: /content/VL1_CAT.zip
100% 761k/761k [00:00<00:00, 132MB/s]


In [ ]:
#!file TS1_CAT.zip

In [4]:
import zipfile

zip_file_path = '/content/VL1_CAT.zip'
extract_to = '/content/CAT/labels_json/val'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("압축 해제 완료!")

압축 해제 완료!


In [5]:
zip_file_path = '/content/VS1_CAT.zip'
extract_to = '/content/CAT/images/val'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("압축 해제 완료!")

압축 해제 완료!


In [6]:
zip_file_path = '/content/TL1_CAT.zip'
extract_to = '/content/CAT/labels_json/train'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("압축 해제 완료!")

압축 해제 완료!


In [7]:
zip_file_path = '/content/TS1_CAT.zip'
extract_to = '/content/CAT/images/train'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("압축 해제 완료!")

압축 해제 완료!


# 고양이 질병 훈련

### json txt/csv 파일변환
- EDA용

In [ ]:
import os
import json

input_folder = '/content/train/T_lavels'
output_file = '/content/train/t_lavel.txt'

with open(output_file, 'w', encoding='utf-8') as out_file:
    for filename in sorted(os.listdir(input_folder)):
        if filename.endswith('.json'):
            file_path = os.path.join(input_folder, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                out_file.write(json.dumps(data, ensure_ascii=False) + '\n')


In [ ]:
import pandas as pd

txt_file_path = "/content/train/t_lavel.txt"

records = []
with open(txt_file_path, "r", encoding="utf-8") as f:
    for line in f:
        entry = json.loads(line)
        metadata = entry.get("metadata", {})
        record = {
            "Patient-id": metadata.get("Birth", ""),
            "Annotation-id": metadata.get("Annotation-id", ""),
            "Sex": metadata.get("Sex", ""),
            "Date": metadata.get("Date", ""),
            "Position": metadata.get("Position", ""),
            "Disease": metadata.get("Disease", ""),
            "Disease-Name": metadata.get("Disease-Name", ""),
            "fileName": entry.get("fileName", "")
        }
        records.append(record)

# 정렬
t_lavel_df = pd.DataFrame(records)
t_lavel_df = t_lavel_df.sort_values(by=["Patient-id"]).reset_index(drop=True)

# CSV 저장
csv_path = "/content/train/t_lavel.csv"
t_lavel_df.to_csv(csv_path, index=False)


### 라벨링된 JSON 파일을 YOLO 포맷 텍스트 파일로 변환

- train/val 각각 result_labels 폴더에 저장됨

In [ ]:
import os
import json
from PIL import Image

# 기본 경로
base_path = "/content/CAT"

# 경로 딕셔너리
sets = {
    'train': {
        'json_folder': os.path.join(base_path, "labels_json", "train"),   # JSON 라벨
        'image_folder': os.path.join(base_path, "images", "train"),  # 이미지
        'output_folder': os.path.join(base_path, "labels", "train")  # YOLO 형식 저장될 위치
    },
    'val': {
        'json_folder': os.path.join(base_path, "labels_json", "val"),
        'image_folder': os.path.join(base_path, "images", "val"),
        'output_folder': os.path.join(base_path, "labels", "val")
    }
}

# 지원 이미지 확장자
image_exts = ['.jpg', '.jpeg', '.png']

# 변환 함수 정의
def convert_to_yolo(json_folder, image_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for file in os.listdir(json_folder):
        if file.endswith('.json'):
            base_name = os.path.splitext(file)[0]

            # JSON 로드
            with open(os.path.join(json_folder, file), 'r', encoding='utf-8') as f:
                data = json.load(f)

            # 이미지 찾기
            image_path = None
            for ext in image_exts:
                candidate = os.path.join(image_folder, base_name + ext)
                if os.path.exists(candidate):
                    image_path = candidate
                    break

            if not image_path:
                print(f"⚠️ 이미지 없음: {base_name}")
                continue

            # 이미지 크기
            with Image.open(image_path) as img:
                img_width, img_height = img.size

            yolo_lines = []
            for ann in data.get("annotations", []):
                if ann.get("shape") == "Bounding Box":
                    (x1, y1), (x2, y2) = ann["points"]
                    cx = (x1 + x2) / 2 / img_width
                    cy = (y1 + y2) / 2 / img_height
                    w = abs(x2 - x1) / img_width
                    h = abs(y2 - y1) / img_height
                    yolo_lines.append(f"0 {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}")

            # 결과 저장
            out_path = os.path.join(output_folder, base_name + ".txt")
            with open(out_path, "w", encoding='utf-8') as f:
                f.write("\n".join(yolo_lines))

            print(f"✅ 변환 완료: {base_name}.txt")

# ▶ 각 세트(train, val)에 대해 실행
for phase, paths in sets.items():
    print(f"\n📂 처리 중: {phase}")
    convert_to_yolo(paths['json_folder'], paths['image_folder'], paths['output_folder'])

In [ ]:
#!mv /content/CAT/images/train/TS1_CAT/* /content/CAT/images/train

### yaml 파일 제작

In [9]:
# YAML 파일 내용
yaml_content = """
train: /content/CAT/images/train
val: /content/CAT/images/val

nc: 1
names: ['cat']
"""

# 'cat.yaml' 파일을 지정된 경로에 생성
with open('/content/CAT/cat.yaml', 'w') as file:
    file.write(yaml_content)

print("cat.yaml 파일 생성완료.")

cat.yaml 파일 생성완료.


## yolo11m 사용

In [10]:
!git clone https://github.com/ultralytics/yolo11

Cloning into 'yolo11'...
fatal: could not read Username for 'https://github.com': No such device or address


In [11]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
!yolo detect train data='/content/CAT/cat.yaml' model=yolo11m.pt epochs=100 imgsz=640 patience=10 batch=-1 seed=42 single_cls=True

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100% 38.8M/38.8M [00:00<00:00, 355MB/s]
Ultralytics 8.3.116 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/content/CAT/cat.yaml, epochs=100, time=None, patience=10, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False

### ❗재시작

In [12]:
!yolo detect train resume=True model='/content/drive/MyDrive/CAT_RUNS/runs/detect/train/weights/last.pt' epochs=100 imgsz=640 patience=10 batch=-1 seed=42 single_cls=True

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.118 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/CAT_RUNS/runs/detect/train/weights/last.pt, data=/content/CAT/cat.yaml, epochs=100, time=None, patience=10, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=/content/drive/MyDrive/CAT_RUNS/runs/detect/train/weights/last.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap

In [ ]:
#!rm -rf /content/runs/detect/train3

In [ ]:
#이어서 학습
!yolo detect train resume=True model='/content/drive/MyDrive/yolo_runs/cat/weights/last.pt'


In [ ]:
#모델 학습 과정 확인
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/CAT_RUNS/runs1/detect/train